In [3]:
import numpy as np
import pandas as pd
import time
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
from tqdm import tqdm

In [4]:
# Загрузка необходимых ресурсов NLTK
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)

True

In [11]:
class CustomLDA:
    def __init__(self, n_topics=10, alpha=0.1, beta=0.01, n_iter=1000, random_state=None):
        self.n_topics = n_topics
        self.alpha = alpha
        self.beta = beta
        self.n_iter = n_iter
        self.random_state = random_state
        self.components_ = None
        self.topic_word_ = None
        self.doc_topic_ = None
        
    def fit(self, X):
        # Инициализация параметров
        n_docs, n_words = X.shape
        random_state = np.random.RandomState(self.random_state)
        
        # Инициализация матриц подсчетов
        n_dk = np.zeros((n_docs, self.n_topics)) + self.alpha  # счетчик, сколько раз тема k встречается в документе d (плюс сглаживание alpha).
        n_kw = np.zeros((self.n_topics, n_words)) + self.beta   # счетчик, сколько раз слово w встречается в теме k (плюс сглаживание beta).
        n_k = np.zeros(self.n_topics) + n_words * self.beta     # сумма слов в теме k (плюс beta * n_words).
        
        # Инициализация тематических назначений
        z = []
        doc_word_indices = []
        
        # Преобразование разреженной матрицы в формат, удобный для обработки
        X_dense = X.toarray()
        
        for d in range(n_docs):
            doc_z = []
            doc_words = []
            for w in range(n_words):
                count = int(X_dense[d, w])
                if count > 0:
                    # Начальное случайное назначение темы
                    topics = random_state.choice(self.n_topics, count, replace=True)
                    doc_z.extend(topics)
                    doc_words.extend([w] * count)
                    
                    # Обновление счетчиков
                    for t in topics:
                        n_dk[d, t] += 1
                        n_kw[t, w] += 1
                        n_k[t] += 1
            z.append(np.array(doc_z))
            doc_word_indices.append(np.array(doc_words))
        
        # Сэмплирование по Гиббсу
        for iteration in tqdm(range(self.n_iter), desc="Gibbs Sampling"):
            for d in range(n_docs):
                word_indices = doc_word_indices[d]
                doc_len = len(z[d])
                
                for i in range(doc_len):
                    w = word_indices[i]
                    t = z[d][i]
                    
                    # Удаление текущего назначения
                    n_dk[d, t] -= 1
                    n_kw[t, w] -= 1
                    n_k[t] -= 1
                    
                    # Расчет вероятностей для нового назначения
                    p_topic = (n_dk[d, :] + self.alpha) * (n_kw[:, w] + self.beta) / (n_k + n_words * self.beta)
                    
                    # Нормализация вероятностей
                    p_topic_sum = p_topic.sum()
                    if p_topic_sum > 0:
                        p_topic /= p_topic_sum
                    else:
                        p_topic = np.ones(self.n_topics) / self.n_topics
                    
                    # Выбор новой темы
                    t = random_state.choice(self.n_topics, p=p_topic)
                    z[d][i] = t
                    
                    # Обновление счетчиков
                    n_dk[d, t] += 1
                    n_kw[t, w] += 1
                    n_k[t] += 1
        
        # Расчет итоговых распределений
        self.components_ = (n_kw + self.beta) / (n_k[:, np.newaxis] + n_words * self.beta)
        self.topic_word_ = self.components_
        self.doc_topic_ = (n_dk + self.alpha) / (n_dk.sum(axis=1)[:, np.newaxis] + self.n_topics * self.alpha)
        
        return self
    
    def transform(self, X):
        n_docs, n_words = X.shape
        random_state = np.random.RandomState(self.random_state)
    
    
        n_dk = np.zeros((n_docs, self.n_topics)) + self.alpha
        n_kw = np.zeros((self.n_topics, n_words)) + self.beta
        n_k = np.zeros(self.n_topics) + n_words * self.beta

        # Преобразование разреженной матрицы в плотную
        X_dense = X.toarray()
    
        # Инициализация тематических назначений
        z = []
        doc_word_indices = []
        for d in range(n_docs):
            doc_z = []
            doc_words = []
            for w in range(n_words):
                count = int(X_dense[d, w])
                if count > 0:
                    topics = random_state.choice(self.n_topics, count, replace=True)
                    doc_z.extend(topics)
                    doc_words.extend([w] * count)
                    for t in topics:
                        n_dk[d, t] += 1
                        n_kw[t, w] += 1
                        n_k[t] += 1
            z.append(np.array(doc_z))
            doc_word_indices.append(np.array(doc_words))

        # Гиббсовское сэмплирование для новых документов
        for iteration in range(self.n_iter // 10):  # Меньше итераций для ускорения
            for d in range(n_docs):
                word_indices = doc_word_indices[d]
                doc_len = len(z[d])
                for i in range(doc_len):
                    w = word_indices[i]
                    t = z[d][i]

                    # Удаление текущего назначения
                    n_dk[d, t] -= 1
                    n_kw[t, w] -= 1
                    n_k[t] -= 1

                    # Расчёт вероятностей
                    p_topic = (n_dk[d, :] + self.alpha) * (n_kw[:, w] + self.beta) / (n_k + n_words * self.beta)
                    p_topic /= p_topic.sum()

                    # Выбор новой темы
                    t = random_state.choice(self.n_topics, p=p_topic)
                    z[d][i] = t

                    # Обновление счетчиков
                    n_dk[d, t] += 1
                    n_kw[t, w] += 1
                    n_k[t] += 1

        return n_dk / n_dk.sum(axis=1)[:, np.newaxis]
    
    def get_top_words(self, feature_names, n_top_words=10):
        top_words = []
        for topic_idx, topic in enumerate(self.components_):
            top_words_indices = topic.argsort()[-n_top_words:][::-1]
            top_words.append([feature_names[i] for i in top_words_indices])
        return top_words

In [5]:
def preprocess_text(text):
    # Удаление спецсимволов и цифр
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    
    # Приведение к нижнему регистру
    text = text.lower()
    
    # Токенизация
    words = text.split()
    
    # Удаление стоп-слов
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words and len(word) > 2]
    
    # Лемматизация
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    
    return words

In [6]:
def load_dataset():
    # Загрузка подмножества данных
    categories = ['sci.space', 'comp.graphics', 'rec.sport.baseball', 'talk.politics.mideast']
    dataset = fetch_20newsgroups(subset='train', categories=categories, 
                                remove=('headers', 'footers', 'quotes'), shuffle=True, random_state=42)
    texts = dataset.data[:100]  # Используем меньше документов для демонстрации
    
    # Предварительная обработка текстов
    processed_texts = [preprocess_text(text) for text in texts]
    
    # Создание матрицы документ-термин
    vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=1000)
    
    # Преобразование в список строк для CountVectorizer
    text_strings = [" ".join(tokens) for tokens in processed_texts]
    X = vectorizer.fit_transform(text_strings)
    feature_names = vectorizer.get_feature_names_out()
    
    return X, feature_names, processed_texts, dataset.target

In [7]:
def visualize_topics(top_words, title="Топ-слова тем"):
    n_topics = len(top_words)
    n_cols = min(2, n_topics)
    n_rows = (n_topics + 1) // n_cols
    
    plt.figure(figsize=(12, 3 * n_rows))
    plt.suptitle(title, fontsize=16)
    
    for topic_idx, words in enumerate(top_words):
        plt.subplot(n_rows, n_cols, topic_idx + 1)
        # Обратный порядок для отображения самого важного слова сверху
        words_sorted = list(reversed(words))
        plt.barh(range(len(words_sorted)), [1] * len(words_sorted), align='center')
        plt.yticks(range(len(words_sorted)), words_sorted)
        plt.title(f"Тема #{topic_idx + 1}")
        plt.gca().invert_yaxis()  # Инвертируем ось Y для правильного отображения
    
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.savefig("lda_topics.png", dpi=300)
    plt.close()

In [8]:
def calculate_coherence(top_words, texts):
    # Создание словаря для gensim
    from gensim.corpora import Dictionary
    gensim_dict = Dictionary(texts)
    corpus = [gensim_dict.doc2bow(text) for text in texts]
    
    # Вычисление когерентности
    coherence_model = CoherenceModel(
        topics=top_words,
        texts=texts,
        dictionary=gensim_dict,
        coherence='c_v'
    )
    return coherence_model.get_coherence()

In [9]:
def get_top_words_from_components(components, feature_names, n_top_words=10):
    top_words = []
    for topic_idx, topic in enumerate(components):
        top_words_indices = topic.argsort()[-n_top_words:][::-1]
        top_words.append([feature_names[i] for i in top_words_indices])
    return top_words

In [15]:
def run_experiment():
    # Загрузка и подготовка данных
    X, feature_names, processed_texts, _ = load_dataset()
    n_topics = 4
    
    # Результаты
    results = {}
    
    # Обучение и оценка собственной реализации LDA
    print("\nОбучение модели: Собственный LDA")
    try:
        custom_lda = CustomLDA(n_topics=n_topics, alpha=0.1, beta=0.01, n_iter=100, random_state=42)
        start_time = time.time()
        custom_lda.fit(X)
        train_time = time.time() - start_time
        
        # Получение топ-слов
        custom_top_words = custom_lda.get_top_words(feature_names, n_top_words=10)
        
        # Оценка когерентности тем
        coherence = calculate_coherence(custom_top_words, processed_texts)
        
        results["Собственный LDA"] = {
            'coherence': coherence,
            'train_time': train_time,
            'model': custom_lda,
            'top_words': custom_top_words
        }
        
        print(f"Собственный LDA завершен. Когерентность: {coherence:.4f}, Время обучения: {train_time:.2f} сек", custom_top_words)
    except Exception as e:
        print(f"Ошибка при обучении Собственный LDA: {str(e)}")
        results["Собственный LDA"] = {
            'coherence': np.nan,
            'train_time': np.nan,
            'model': None,
            'top_words': []
        }
     # Обучение и оценка реализации sklearn LDA
    print("\nОбучение модели: Sklearn LDA")
    try:
        sklearn_lda = LatentDirichletAllocation(
            n_components=n_topics, 
            doc_topic_prior=0.1, 
            topic_word_prior=0.01,
            learning_method='batch',
            max_iter=10,
            random_state=42
        )
        start_time = time.time()
        sklearn_lda.fit(X)
        train_time = time.time() - start_time
        
        # Получение топ-слов
        sklearn_top_words = get_top_words_from_components(sklearn_lda.components_, feature_names, n_top_words=10)
        
        # Оценка когерентности тем
        coherence = calculate_coherence(sklearn_top_words, processed_texts)
        
        results["Sklearn LDA"] = {
            'coherence': coherence,
            'train_time': train_time,
            'model': sklearn_lda,
            'top_words': sklearn_top_words
        }
        print(f"Sklearn LDA завершен. Когерентность: {coherence:.4f}, Время обучения: {train_time:.2f} сек", sklearn_top_words)
    except Exception as e:
        print(f"Ошибка при обучении Sklearn LDA: {str(e)}")
        results["Sklearn LDA"] = {
            'coherence': np.nan,
            'train_time': np.nan,
            'model': None,
            'top_words': []
        }
    
    return results, processed_texts, feature_names

In [16]:
if __name__ == "__main__":
    try:
        results, processed_texts, feature_names = run_experiment()
        
        # Визуализация тем
        for name, res in results.items():
            if res.get('top_words'):
                visualize_topics(res['top_words'], title=f"Топ-слова тем ({name})")
        
    except Exception as e:
        print(f"Критическая ошибка при выполнении эксперимента: {str(e)}")
        results = {}
        processed_texts = []


Обучение модели: Собственный LDA


Gibbs Sampling: 100%|████████████████████████████████████████████████████████████████| 100/100 [01:08<00:00,  1.45it/s]


Собственный LDA завершен. Когерентность: 0.5620, Время обучения: 69.15 сек [['space', 'list', 'istanbul', 'post', 'group', 'system', 'email', 'information', 'presentation', 'option'], ['one', 'time', 'last', 'year', 'many', 'three', 'day', 'back', 'mean', 'thing'], ['armenian', 'azerbaijan', 'people', 'said', 'armenia', 'turkey', 'dead', 'turkish', 'village', 'town'], ['would', 'dont', 'like', 'get', 'know', 'could', 'however', 'team', 'see', 'think']]

Обучение модели: Sklearn LDA
Sklearn LDA завершен. Когерентность: 0.5563, Время обучения: 0.71 сек [['armenian', 'istanbul', 'people', 'turkey', 'dead', 'said', 'turkish', 'like', 'one', 'new'], ['space', 'list', 'post', 'one', 'group', 'would', 'system', 'presentation', 'information', 'email'], ['armenian', 'azerbaijan', 'people', 'azerbaijani', 'attack', 'region', 'april', 'refugee', 'government', 'armenia'], ['option', 'philadelphia', 'would', 'power', 'dont', 'also', 'like', 'module', 'team', 'used']]
